In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio autoplay="" src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" loop controls>

In [ ]:
#@title Install Backend & Deps
!apt update
!apt install aria2 -y

!echo Downloading KoboldCpp, please wait...
!wget -O dlfile.tmp https://kcpplinux.concedo.workers.dev && mv dlfile.tmp koboldcpp_linux
!test -f koboldcpp_linux && echo Download Successful || echo Download Failed

In [ ]:
#@title Download Model
OPTIONS = {
    1: {
        "id": "mradermacher/Fimbulvetr-11B-v2-GGUF",
        "file": "https://huggingface.co/mradermacher/Fimbulvetr-11B-v2-GGUF/blob/main/Fimbulvetr-11B-v2.Q5_K_S.gguf"
    },
    2: {
        "id": "mradermacher/MN-LooseCannon-12B-v2-GGUF",
        "file": "https://huggingface.co/mradermacher/MN-LooseCannon-12B-v2-GGUF/blob/main/MN-LooseCannon-12B-v2.Q6_K.gguf"
    },
    3: {
        "id": "bartowski/Dans-PersonalityEngine-V1.1.0-12b-GGUF",
        "file": "https://huggingface.co/bartowski/Dans-PersonalityEngine-V1.1.0-12b-GGUF/blob/main/Dans-PersonalityEngine-V1.1.0-12b-Q6_K.gguf"
    },
    4: {
        "id":"mradermacher/Magnum-Picaro-0.7-v2-12b-i1-GGUF",
        "file":"https://huggingface.co/mradermacher/Magnum-Picaro-0.7-v2-12b-i1-GGUF/blob/main/Magnum-Picaro-0.7-v2-12b.i1-Q6_K.gguf"
    }
}

PRESET = "1" #@param [1,2,3,4]

MODEL = OPTIONS[int(PRESET)]["file"].replace("/blob/", "/resolve/")

!aria2c -x 10 -o model.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $MODEL

In [ ]:
#@LAUNCH!

GPU_LAYERS = 80 #@param
CONTEXT_SIZE = 8192 #@param

!chmod +x ./koboldcpp_linux
!./koboldcpp_linux model.gguf --noshift --usecublas 0 mmq --gpulayers $GPU_LAYERS --contextsize $CONTEXT_SIZE --quiet --remotetunnel